In [2]:
import cv2
import numpy as np

from sklearn.metrics import pairwise

In [3]:
backGround = None
accumulated_weight = 0.5

roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [4]:
def calc_accum_avg(frame,accumulated_weight):
    
    global backGround
    
    if backGround is None:
        backGround = frame.copy().astype('float')
        return None
    
    cv2.accumulated_weight(frame,backGround,accumulated_weight)